- Create a new DNN that reuses all the pre-trained hidden layers of the previous model, freezes them, and replaces the softmax output layer with a fresh new one. 
- Train this new DNN on digits 5 to 9, using only 100 images per digit, and time how long it takes. Despite this small number of examples, can you achieve high precision? 
- Try caching the frozen layers, and train the model again: how much faster is it now? 
- Try again reusing just four hidden layers instead of five. Can you achieve a higher precision? 
- Now unfreeze the top two hidden layers and continue training: can you get the model to perform even better?

### Get Data

In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28)

train_indxes_5to9 = y_train>=5
train_x_5to9 = x_train[train_indxes_5to9]
train_y_5to9 = y_train[train_indxes_5to9]-5


def get_random_indxs(y_train, group_value, count=10):
    train_indxs = np.arange(len(y_train), dtype=np.int32)
    group_indxs = train_indxs[y_train == group_value]
    indxs = np.random.choice(group_indxs,count)
    return indxs

def get_samples_of_each_group(y_train, count=150):
    rand_indxs = np.array([], dtype=np.int32)
    for group_val in np.unique(y_train):
        rand_indxs=np.r_[rand_indxs,get_random_indxs(y_train,group_val, count)]
        
    np.random.shuffle(rand_indxs)
    return rand_indxs

rand_indxs = get_samples_of_each_group(train_y_5to9)
train_x_5to9 = train_x_5to9[rand_indxs]
train_y_5to9 = train_y_5to9[rand_indxs]


train_x_5to9, val_x_5to9, train_y_5to9, val_y_5to9 = \
    train_test_split(train_x_5to9, train_y_5to9, test_size=.3, stratify=train_y_5to9)

val_x_5to9, test_x_5to9, val_y_5to9, test_y_5to9 = \
    train_test_split(val_x_5to9,val_y_5to9, test_size=.6, stratify=val_y_5to9)

/Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [43]:
train_test_split?

### Verify that DNN_Classifier is not broken

In [71]:
# from imp import reload
# import my_libs
# reload(my_libs.dnn)
from my_libs.dnn import DNN_Classifier

classifier = DNN_Classifier(n_hidden_layers=3, n_neurons=100,n_outputs=5,
                            batch_norm_momentum=.95)

classifier.fit(train_x_5to9, train_y_5to9, val_x_5to9, val_y_5to9)

epoch 0, score 0.900000, loss 0.117199
epoch 20, score 0.955556, loss 0.001046
epoch 40, score 0.933333, loss 0.002180
epoch 60, score 0.900000, loss 0.014893
epoch 80, score 0.955556, loss 0.013935
epoch 100, score 0.922222, loss 0.000119
epoch 120, score 0.933333, loss 0.000265
epoch 140, score 0.933333, loss 0.000468
epoch 160, score 0.966667, loss 0.000082
epoch 180, score 0.944444, loss 0.000013
epoch 200, score 0.944444, loss 0.000055
epoch 220, score 0.966667, loss 0.000005
epoch 240, score 0.922222, loss 0.000300
epoch 260, score 0.922222, loss 0.000110
No progress for 200 epoches.
Reverting back to epoch 73                     with 0.977778 score


DNN_Classifier(activation=<function elu at 0x1a24f7d268>,
        batch_norm_momentum=0.95, batch_size=50, dropout_rate=None,
        learning_rate=0.01, n_hidden_layers=3, n_neurons=100, n_outputs=5,
        optimizer=<class 'tensorflow.python.training.adam.AdamOptimizer'>)

### Create DNN_Classifier_Transfer
which will transfer learning of DNN_Classifier from saved file

In [2]:
# from imp import reload
# import my_libs
# reload(my_libs.dnn)

import tensorflow as tf
from my_libs.dnn import DNN_Classifier, get_optimizer_op, get_validation_score, \
    get_softmax_xentropy_loss
from tensorflow.train import AdamOptimizer

class DNN_Classifier_Transfer(DNN_Classifier):
    def __init__(self, checkpoint_name, use_hidden_layers=0):
        DNN_Classifier.__init__(self, batch_size=20)
        self._checkpoint_name = checkpoint_name
        self._batch_norm_update_ops = None
        self.use_hidden_layers = use_hidden_layers
        
    
    def _initialize_session_and_graph(self):
        DNN_Classifier._restore_graph(self,self._checkpoint_name)
        
        self.restore_n_hidden_layers()
        
        get_layer_name = lambda i: "hidden%d"%(self.n_hidden_layers-i)
        variable_scopes = [get_layer_name(i) for i in range(self.use_hidden_layers)]
        
        trainable_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 
                                                scope="output")
        
        for scope in variable_scopes:
            trainable_variables.append(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 
                                                scope=scope))
        
        optimizer = AdamOptimizer(0.01, name="adam2")
        self._optimizer_op = optimizer.minimize(self._loss, var_list=trainable_variables)
        
        DNN_Classifier._restore_session(self, self._checkpoint_name)
        return self._graph
    
    def restore_n_hidden_layers(self):
        i=1
        tensors = []
        while True:
            try:       
                tensor = self._graph.get_tensor_by_name("DNN/hiden%d_out:0"%i)
                tensors.append(tensor)
                i+=1
            except:
                break
        self.n_hidden_layers = i-1
        
        

W0827 19:36:31.123486 4489565632 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:73: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



In [3]:
# np.random.seed(5)
# tf.random.set_random_seed(5)

transfer_classifier = DNN_Classifier_Transfer("Mnist-0to4-best_batch_norm", use_hidden_layers=2)
transfer_classifier.fit(train_x_5to9, train_y_5to9, val_x_5to9, val_y_5to9)


W0827 19:36:48.534260 4489565632 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:223: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0827 19:36:48.538044 4489565632 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:224: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W0827 19:36:49.280345 4489565632 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:225: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 19:36:49.343497 4489565632 deprecation.py:323] From /Users/devbhadurkhadka/.pyenv/versions/anaconda3-5.2.0/envs/scikit_practice/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.a

epoch 0, score 0.311111, loss 16.103727
epoch 20, score 0.488889, loss 1.239170
epoch 40, score 0.466667, loss 1.082394
epoch 60, score 0.444444, loss 1.059728
epoch 80, score 0.477778, loss 1.024295
epoch 100, score 0.488889, loss 1.236668
epoch 120, score 0.511111, loss 0.656108
epoch 140, score 0.500000, loss 0.786920
epoch 160, score 0.555556, loss 0.781088
epoch 180, score 0.488889, loss 0.497093
epoch 200, score 0.433333, loss 0.188531
epoch 220, score 0.377778, loss 0.859469
epoch 240, score 0.277778, loss 0.543417
epoch 260, score 0.266667, loss 0.434174
epoch 280, score 0.266667, loss 0.394407
epoch 300, score 0.288889, loss 0.173341
epoch 320, score 0.211111, loss 0.562680
epoch 340, score 0.266667, loss 0.412937
No progress for 200 epoches.
Reverting back to epoch 156                     with 0.588889 score


DNN_Classifier_Transfer(checkpoint_name=None, use_hidden_layers=2)

### Crate DNN_Classifier_Frozen
- Same as DNN_Classifier_Transfer but it will pre calculate output of previous layer for all data

In [64]:
from imp import reload
import my_libs
reload(my_libs.dnn)
from my_libs.dnn import DNN_Classifier

class DNN_Classifier_Frozen(DNN_Classifier_Transfer):
        
    def _initialize_session_and_graph(self):
        DNN_Classifier_Transfer._initialize_session_and_graph(self)
        forzen_output_name = "DNN/hiden%d_out:0"%(self.n_hidden_layers-self.use_hidden_layers)
        
        self._frozen_out = self._graph.get_tensor_by_name(forzen_output_name)
        ## replace input placeholder _x with output of hidden layer _frozen_out
        self._old_x = self._x
        self._x = self._frozen_out
    
    
    def fit(self, x, y, val_x, val_y):
        self._initialize_session_and_graph()
        with self._session.as_default() as sess:
            self._x_frozen_out = self._frozen_out.eval(session=sess,feed_dict={self._old_x:x})
            self._val_x_frozen_out = self._frozen_out.eval(feed_dict={self._old_x:val_x})
            
        return DNN_Classifier.fit(self, self._x_frozen_out, y, self._val_x_frozen_out, val_y)
    

In [68]:
np.random.seed(0)
tf.random.set_random_seed(0)

frozen_classifier = DNN_Classifier_Frozen("Mnist-0to4-best_batch_norm", use_hidden_layers=2)
frozen_classifier.fit(train_x_5to9, train_y_5to9, val_x_5to9, val_y_5to9)

epoch 0, score 0.422222, loss 1.747186
epoch 50, score 0.233333, loss 0.270999
epoch 100, score 0.133333, loss 0.422296
epoch 150, score 0.200000, loss 0.510700
epoch 200, score 0.211111, loss 0.245352
No progress for 200 epoches.
Reverting back to epoch 0                     with 0.422222 score


DNN_Classifier_Frozen(checkpoint_name=None, use_hidden_layers=2)

### Test if running variable initializer resets previously set value

In [132]:
tf.reset_default_graph()
xx = tf.Variable(0, dtype=np.int16, name="xx")
inc = tf.assign(xx, xx+1)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        sess.run(inc)
    sess.run(init)
    print(xx.eval())
    

0


In [5]:
transfer_classifier = DNN_Classifier_Transfer("Mnist-0to4-best_batch_norm")
graph=transfer_classifier._initialize_session_and_graph()

W0827 09:49:44.197348 4536214976 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:223: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0827 09:49:44.199877 4536214976 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:224: The name tf.train.import_meta_graph is deprecated. Please use tf.compat.v1.train.import_meta_graph instead.

W0827 09:49:45.264413 4536214976 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:225: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 09:49:45.501465 4536214976 deprecation_wrapper.py:119] From /Volumes/Projects/Machine Learning/tensorflow_practice/my_libs/dnn.py:240: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0827 09:49:45.504031 4536214976 deprecation_wrapper

In [58]:
[op.name for op in graph.get_operations() if ptrn.search(op.name)]

['DNN/hiden1_out',
 'DNN/hiden2_out',
 'DNN/hiden3_out',
 'DNN/hiden4_out',
 'DNN/hiden5_out']

In [57]:
import re

ptrn=re.compile(r"hiden\d+_out$")
print(ptrn.search("DNN/hiden5_out"))

<_sre.SRE_Match object; span=(4, 14), match='hiden5_out'>


In [67]:
def get_n_hidden_layers(graph):
    i=1
    tensors = []
    while True:
        try:       
            tensor = graph.get_tensor_by_name("DNN/hiden%d_out:0"%i)
            tensors.append(tensor)
            i+=1
        except:
            break
    return i-1

get_n_hidden_layers(graph)

5

In [72]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 
                                                scope="hidden5")

[<tf.Variable 'hidden5/kernel:0' shape=(150, 150) dtype=float32_ref>,
 <tf.Variable 'hidden5/bias:0' shape=(150,) dtype=float32_ref>]